# 완성된 Transformer 모델
```Plain Text
Transformer/
├── main.py               # main 파일 (테스트용)
└── modules/
    ├── MyTransformer.py  # 인코더-디코더
    └── common.py         # 여러 번 재사용되는 구조들
                          ## attention, Embedding, MultiHeadAttention, FeedForward
```

In [ ]:
class Config:
    def __init__(self, vocab_size, pad_token_id=0):
        self.vocab_size = vocab_size        # 단어사전 크기
        self.hidden_size = 768              # 은닉층 차원수 (attention_eads의 배수)
        self.max_position_embeddings = 768  # 최대 시퀀스 길이
        self.num_attention_heads = 12       # 어텐션 헤드 개수
        self.num_hidden_layers = 12         # 레이어 개수
        self.intermediate_size = 2048       # FeedForward 레이어의 중간 차원
        self.hidden_dropout_prob = 0.1      # dropout 비율
        self.layer_norm_eps = 1e-12         # epsilon
        self.pad_token_id = pad_token_id    # 패딩 토큰

In [3]:
# 소스와 타겟의 단어사전 크기가 다르다고 가정 (번역기 등)
SOURCE_VOCAB_SIZE = 10000
TARGET_VOCAB_SIZE = 15000

PAD_ID = 0

In [ ]:
import torch
from modules.MyTransformer import Transformer

config_enc = Config(vocab_size=SOURCE_VOCAB_SIZE, pad_token_id=PAD_ID)
config_dec = Config(vocab_size=TARGET_VOCAB_SIZE, pad_token_id=PAD_ID)

model = Transformer(config_enc, config_dec)
model.eval() # 평가 모드

# 더미 입력 데이터
# (batch_size=2, source_seq_len=12, target_seq_len=10)
source_ids = torch.tensor([
    [101, 2054, 2064, 2106, 102, 0, 0, 0, 0, 0, 0, 0],
    [101, 3000, 4000, 102, 2000, 102, 0, 0, 0, 0, 0, 0]
], dtype=torch.long)

target_ids = torch.tensor([
    [201, 3054, 3064, 3106, 202, 0, 0, 0, 0, 0],
    [201, 4000, 5000, 202, 6000, 202, 0, 0, 0, 0]
], dtype=torch.long)

print(f"Source IDs shape: {source_ids.shape}")
print(f"Target IDs shape: {target_ids.shape}")

# 모델이 내부적으로 모든 마스크를 생성하여 처리
logits = model(source_ids, target_ids)
print(f"Output Logits shape: {logits.shape}")

# (batch_size, target_seq_len, target_vocab_size)
# 예상 출력: [2, 10, 15000]

Source IDs shape: torch.Size([2, 12])
Target IDs shape: torch.Size([2, 10])
Output Logits shape: torch.Size([2, 10, 15000])
